## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/BootCamp_DataAnalytics/Module_challenges/Challenge-6/Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Jamestown,US,42.0970,-79.2353,19.77,55,75,6.17,broken clouds
1,Tuktoyaktuk,CA,69.4541,-133.0374,-15.00,72,75,5.66,broken clouds
2,Provideniya,RU,64.3833,-173.3000,-6.80,91,100,8.34,overcast clouds
3,Kailua,US,21.4022,-157.7394,23.41,83,100,6.91,overcast clouds
4,Cidreira,BR,-30.1811,-50.2056,21.75,79,94,4.19,overcast clouds
5,Altamira,BR,-3.2033,-52.2064,29.05,74,75,2.57,broken clouds
6,Kavaratti,IN,10.5669,72.6420,29.39,69,100,2.93,heavy intensity rain
7,Srivardhan,IN,18.0333,73.0167,28.67,63,78,1.32,broken clouds
8,Erie,US,42.7667,-78.6664,21.26,64,59,4.02,broken clouds
9,Guerrero Negro,MX,27.9769,-114.0611,13.50,72,0,2.66,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 18
What is the maximum temperature you would like for your trip? 22


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Jamestown,US,42.0970,-79.2353,19.77,55,75,6.17,broken clouds
4,Cidreira,BR,-30.1811,-50.2056,21.75,79,94,4.19,overcast clouds
8,Erie,US,42.7667,-78.6664,21.26,64,59,4.02,broken clouds
17,San Rafael,AR,-34.6177,-68.3301,18.91,27,0,1.60,clear sky
18,Pisco,PE,-13.7000,-76.2167,20.03,73,0,2.57,clear sky
22,Tunduma,TZ,-9.3000,32.7667,20.03,83,72,0.80,moderate rain
24,Hermanus,ZA,-34.4187,19.2345,21.56,84,100,5.04,overcast clouds
48,Gat,IL,31.6100,34.7642,21.89,18,95,5.66,overcast clouds
49,Hilo,US,19.7297,-155.0900,21.82,89,75,1.79,broken clouds
53,Bredasdorp,ZA,-34.5322,20.0403,21.99,66,100,6.22,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
clean_df.count()

City                   152
Country                152
Lat                    152
Lng                    152
Max Temp               152
Humidity               152
Cloudiness             152
Wind Speed             152
Current Description    152
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,19.77,broken clouds,42.0970,-79.2353,
4,Cidreira,BR,21.75,overcast clouds,-30.1811,-50.2056,
8,Erie,US,21.26,broken clouds,42.7667,-78.6664,
17,San Rafael,AR,18.91,clear sky,-34.6177,-68.3301,
18,Pisco,PE,20.03,clear sky,-13.7000,-76.2167,
22,Tunduma,TZ,20.03,moderate rain,-9.3000,32.7667,
24,Hermanus,ZA,21.56,overcast clouds,-34.4187,19.2345,
48,Gat,IL,21.89,overcast clouds,31.6100,34.7642,
49,Hilo,US,21.82,broken clouds,19.7297,-155.0900,
53,Bredasdorp,ZA,21.99,overcast clouds,-34.5322,20.0403,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found - Skip")
        

Hotel not found - Skip
Hotel not found - Skip
Hotel not found - Skip
Hotel not found - Skip
Hotel not found - Skip


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"].astype(bool)]
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,19.77,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
4,Cidreira,BR,21.75,overcast clouds,-30.1811,-50.2056,Hotel Castelo
8,Erie,US,21.26,broken clouds,42.7667,-78.6664,Roycroft Inn
17,San Rafael,AR,18.91,clear sky,-34.6177,-68.3301,Hotel San Rafael
18,Pisco,PE,20.03,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
22,Tunduma,TZ,20.03,moderate rain,-9.3000,32.7667,Sacse High Class Hotel
24,Hermanus,ZA,21.56,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
48,Gat,IL,21.89,overcast clouds,31.6100,34.7642,OlusHome
49,Hilo,US,21.82,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
53,Bredasdorp,ZA,21.99,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor


In [11]:
# 8a. Create the output File (CSV)
output_data_file="C:/BootCamp_DataAnalytics/Module_challenges/Challenge-6/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and Temp Max {Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))